In [114]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [115]:
#(hardware name, number of nodes)
filesD = {
         'knl_mpi1':('knl',1), 'knl_mpi2':('knl',2), 'knl_mpi4':('knl',4),
         'skl_mpi1':('skl',1), 'skl_mpi2':('skl',2), 'skl_mpi4':('skl',4),
         'i5':('i5',1),
         'p100_mpi1':('p100',1), 'p100_mpi2':('p100',2), 'p100_mpi4':('p100',4),
         'v100_mpi1':('v100',1), 'v100_mpi2':('v100',2), 'v100_mpi4':('v100',4),
         'gtx1060':('gtx1060',1)
                      }

files = odict(sorted(filesD.items(), key= lambda t : t[1][1]))

In [116]:
pd.set_option('precision',1)
names={'axpby':3,'dot':2,'dx':3, 'dy':3}
ns=[3,4]
values = []
for f, v in files.items() :#{'knl_mpi2':('knl',2)}.items():
    df=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
    #add size and bandwidth columns
    df.insert(0,'size', 8*df['n']*df['n']*df['Nx']*df['Ny']/1e6/v[1]) #inplace transformation
    for name,memops in names.items() :
        df.insert(0,name+'_bw',df['size']/1000*memops/df[name])
    #compute one version with aggregated grouped sizes and one without
    agg=df.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    agg=agg.reset_index(level=['n','Nx','Ny','size'])
    agg.sort_values(by='size',inplace=True) #sort by size
    df.sort_values(by='size',inplace=True)
    #compute average and average bandwidth among dx and dy
    dxdy = df.assign( dxdy=(df['dx']+df['dy'])/2)
    dxdy = dxdy.assign( dxdy_bw=2.0*df['dx_bw']*df['dy_bw']/(df['dx_bw']+df['dy_bw']))
    #take ns
    dxdy=dxdy.set_index('n')
    dxdy=dxdy.loc[ns,:]
    dxdy.reset_index(inplace=True)
    dxdy.sort_values(by='size',inplace=True)
    #compute one version with aggregated grouped sizes and one without
    aggdxdy=dxdy.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    aggdxdy=aggdxdy.reset_index(level=['n','Nx','Ny','size'])
    aggdxdy.sort_values(by='size',inplace=True) #sort by size
    #aggdxdy[aggdxdy['size']>9]

    
    n =3
    s =30
    mb=9

    line = []
    l=len(df)
    line.append(v[0])
    line.append(v[1])
    line.append(agg[0:n][('axpby','mean')].mean()/1e-6)
    line.append(agg[0:n][('axpby','mean')].min()/1e-6)
    line.append(df[l-s:l]['axpby_bw'].mean())
    line.append(df[l-s:l]['axpby_bw'].std())
    line.append(agg[0:n][('dot','mean')].mean()/1e-6)
    line.append(agg[0:n][('dot','mean')].min()/1e-6)
    line.append(df[l-s:l]['dot_bw'].mean())
    line.append(df[l-s:l]['dot_bw'].std())
    line.append(aggdxdy[0:n][('dxdy','mean')].mean()/1e-6)
    line.append(aggdxdy[0:n][('dxdy','mean')].min()/1e-6)
    line.append(dxdy[dxdy['size']>mb]['dxdy_bw'].mean())
    line.append(dxdy[dxdy['size']>mb]['dxdy_bw'].std())
    values.append(line)
      
tuples=[('arch','',''),('nodes','','')]        
for n in ['axpby','dot','dxdy']:
    tuples.append((n,'lat','avg'))
    tuples.append((n,'lat','min'))
    tuples.append((n,'bw','avg'))
    tuples.append((n,'bw','std'))
cols=pd.MultiIndex.from_tuples(tuples)
arr = pd.DataFrame(values,index=files.keys(), columns=cols)
arr.sort_values(by='arch',inplace=True)
arr.set_index(['arch','nodes'],inplace=True)
#arr.loc[:,[('dot','bw','avg'),('dot','lat','avg')]]
#arr

In [117]:
#arr=arr.reset_index()

In [118]:
def toString(x): 
    string = '%.1f'% x
    return string
addto = []
for n in ['axpby','dot','dxdy']:
    arr.loc[:,(n,'bw','string')]= arr[n]['bw']['avg'].apply(toString) +" ± "+arr[n]['bw']['std'].apply(toString)
    addto.append((n,'lat','min'))
    addto.append((n,'bw','string'))
nicetable=arr[addto]
drop = nicetable.columns.droplevel(2)
nicetable.columns=drop
#nicetable.reset_index(inplace=True)
#nicetable.set_index('arch')
newindex=[('i5',1)]
for n in ['skl','knl']:
    for m in [1,2,4]:
        newindex.append((n,m))
newindex.append(('gtx1060',1))
for n in ['p100','v100']:
    for m in [1,2,4]:
        newindex.append((n,m))
    
nicetable.reindex(newindex)

axpby                  dot                 dxdy               
                lat            bw    lat            bw    lat             bw
arch    nodes                                                               
i5      1      12.4    30.0 ± 0.2  117.4     9.3 ± 0.0   69.8     27.3 ± 2.9
skl     1       6.1   206.8 ± 3.0   24.5   193.6 ± 8.6   50.6   123.2 ± 27.6
        2       6.9   210.4 ± 6.1   32.6   184.5 ± 8.4   60.5   115.4 ± 27.1
        4       6.0  225.5 ± 12.6   40.7  173.0 ± 12.3   51.2   111.8 ± 27.1
knl     1      14.4  447.0 ± 14.8   67.3   143.3 ± 2.2   84.3   134.8 ± 43.6
        2      14.9   453.4 ± 8.9   93.9   128.9 ± 8.2  139.2    92.5 ± 29.2
        4      13.8  450.0 ± 26.2  128.5   112.9 ± 5.8  171.1    87.1 ± 18.0
gtx1060 1       3.5   157.0 ± 0.1  131.6    26.5 ± 0.1   34.5    97.5 ± 16.9
p100    1       3.1   552.8 ± 1.0   56.4   345.8 ± 1.9   15.7   218.7 ± 18.9
        2       3.3   554.2 ± 0.1   56.1   338.2 ± 3.2  473.1   118.5 ± 49.6
        4       3.1   554.8 ± 1.7   68.0   320.3 ± 7.5  678.0    93.0 ± 37.7
v100    1       3.1   848.3 ± 0.7   41.5   591.6 ± 2.9    7.8   681.1 ± 36.6
        2       3.1   847.9 ± 0.4   48.2   575.3 ± 7.0  452.2  235.4 ± 146.3
        4       3.4   844.9 ± 1.8   59.2  525.7 ± 13.9  691.4   147.9 ± 85.3